In [26]:
import math
import torch
import torch.nn.functional as F
import numpy as np


def fisheye_to_equirectangular_torch(
    img_bgr_uint8,  # torch.uint8, [Hf, Wf, 3] 或 [Hf, Wf]
    inv_poly_param,  # OCamCalib inv_polynomial 系数列表 (θ -> ρ)
    affine_param,  # [c, d, e, xc, yc]
    We=2048,
    He=1024,  # 目标 equirectangular 分辨率，通常 He = We/2
    yaw_range=(-math.pi, math.pi),  # 经度范围（水平）：默认全 360°
    pitch_range=(-math.pi / 2, math.pi / 2),  # 纬度范围（垂直）：默认 -90°..+90°
    padding_mode="zeros",  # 超界填充：'zeros'|'border'|'reflection'
):
    """
    将单目鱼眼图（OCam 模型）重映射为 equirectangular 全景图。
    返回: torch.uint8 的全景图 [He, We, 3] 或 [He, We]。
    """
    # ---- 0) 输入张量 & 设备 ----
    if img_bgr_uint8.dim() == 2:
        Hf, Wf = img_bgr_uint8.shape
        C = 1
        img = img_bgr_uint8.unsqueeze(0).unsqueeze(0).float()  # [1,1,Hf,Wf]
    else:
        Hf, Wf, _ = img_bgr_uint8.shape
        C = 3
        img = img_bgr_uint8.permute(2, 0, 1).unsqueeze(0).float()  # [1,3,Hf,Wf]

    device = img.device
    dtype = img.dtype

    # ---- 1) 为目标 equirect 生成经纬网格 (lon, lat) ----
    # 水平 u -> 经度 lon ∈ [yaw_min, yaw_max]
    # 垂直 v -> 纬度 lat ∈ [pitch_min, pitch_max]，注意 v 向下增大，因此要“从上往下”插值
    yaw_min, yaw_max = float(yaw_range[0]), float(yaw_range[1])
    pit_min, pit_max = float(pitch_range[0]), float(pitch_range[1])

    u = torch.linspace(0, We - 1, We, device=device, dtype=dtype)
    v = torch.linspace(0, He - 1, He, device=device, dtype=dtype)
    uu, vv = torch.meshgrid(u, v, indexing="xy")  # [He, We]

    lon = yaw_min + uu * (yaw_max - yaw_min) / max(We - 1, 1)  # [-π, π]
    lat = pit_max - vv * (pit_max - pit_min) / max(He - 1, 1)  # [+π/2 .. -π/2] 从上到下

    # ---- 2) 经纬 -> 摄像机坐标系下的单位方向 (x,y,z) ----
    # 约定：相机坐标 z 前方、x 右、y 下（与像素 v 同向）
    # 用这样的定义：前方 (lon=0, lat=0) -> (0,0,1)
    # 顶部 (lat=+90°)        -> (0,-1,0)
    # 右侧 (lon=+90°, lat=0) -> (1,0,0)
    cos_lat = torch.cos(lat)
    sin_lat = torch.sin(lat)
    sin_lon = torch.sin(lon)
    cos_lon = torch.cos(lon)

    dx = cos_lat * sin_lon
    dy = -sin_lat  # 注意负号：lat 向上为正，而像素 y 向下为正
    dz = cos_lat * cos_lon

    # ---- 3) OCamCalib 投影：方向 -> 鱼眼像素 (u_f, v_f) ----
    eps = 1e-6
    M = torch.sqrt(dx * dx + dy * dy)
    M_safe = torch.clamp(M, min=eps)
    theta = -torch.atan2(
        dz, M_safe
    )  # 与你给的函数一致：theta = -atan2(z, sqrt(x^2+y^2))

    # ρ(θ) 用 Horner 法稳定计算
    inv_poly = torch.tensor(inv_poly_param, dtype=dtype, device=device)
    rho = torch.zeros_like(theta)
    for a in reversed(inv_poly):
        rho = rho * theta + a

    # 单位化平面方向 * ρ
    x_dir = dx / M_safe * rho
    y_dir = dy / M_safe * rho

    # 仿射 + 主点
    c, d, e, xc, yc = [float(t) for t in affine_param]
    u_f = c * x_dir + d * y_dir + xc
    v_f = e * x_dir + y_dir + yc

    # ---- 4) grid_sample 采样需要把 (u_f, v_f) 归一化到 [-1,1] ----
    grid_x = (u_f / (Wf - 1)) * 2.0 - 1.0
    grid_y = (v_f / (Hf - 1)) * 2.0 - 1.0
    grid = torch.stack([grid_x, grid_y], dim=-1).unsqueeze(0)  # [1,He,We,2]

    # ---- 5) 从鱼眼图采样得到全景 ----
    pano = F.grid_sample(
        img, grid, mode="bilinear", padding_mode=padding_mode, align_corners=True
    )  # [1,C,He,We]

    # ---- 6) 输出 uint8 HxWxC ----
    pano = torch.clamp(pano, 0, 255).round().byte()
    if C == 1:
        return pano.squeeze(0).squeeze(0)  # [He, We]
    else:
        return pano.squeeze(0).permute(1, 2, 0)  # [He, We, 3]


# ---------------- 使用示例 ----------------
if __name__ == "__main__":
    import cv2

    inv_poly_param = [
        889.91582358,
        528.72413142,
        -20.52846684,
        67.73730056,
        48.02888563,
        -14.14051173,
        20.22746961,
        38.15325087,
        -6.98641569,
        -26.68100659,
        -12.41222046,
        -1.83261524,
    ]
    affine_param = [
        9.99949231e-01,
        -4.44517298e-04,
        6.30993143e-04,
        9.68732891e02,
        7.48514758e02,
    ]

    # 读入你的 1920x1536 鱼眼图（BGR）
    fisheye = cv2.imread("./assets/SurCam01.jpeg", cv2.IMREAD_COLOR)
    assert fisheye is not None and fisheye.shape[1] == 1920 and fisheye.shape[0] == 1536

    # -> torch
    img_t = torch.from_numpy(fisheye)  # [Hf,Wf,3], uint8

    # 生成 360x180 的 equirect（注意：背面会是黑的，因为单目看不到）
    pano = (
        fisheye_to_equirectangular_torch(
            img_t,
            inv_poly_param,
            affine_param,
            We=2048,
            He=1024,  # 典型比例：He = We/2
            yaw_range=(-math.pi, math.pi),  # 全 360°
            pitch_range=(-math.pi / 2, math.pi / 2),  # -90°..+90°
            padding_mode="zeros",
        )
        .cpu()
        .numpy()
    )

    cv2.imwrite("pano_equirect_2048x1024.png", pano)
    print("saved -> pano_equirect_2048x1024.png")

    # 如果你只想要前半球（避免黑边），把 yaw 限制到 180°：
    pano_front = (
        fisheye_to_equirectangular_torch(
            img_t,
            inv_poly_param,
            affine_param,
            We=2048,
            He=1024,
            yaw_range=(-math.pi / 2, math.pi / 2),  # 只取 -90°..+90° 水平
            pitch_range=(-math.pi / 2, math.pi / 2),  # 垂直仍是 -90°..+90°
        )
        .cpu()
        .numpy()
    )
    cv2.imwrite("pano_front_hemi_2048x1024.png", pano_front)
    print("saved -> pano_front_hemi_2048x1024.png")

saved -> pano_equirect_2048x1024.png
saved -> pano_front_hemi_2048x1024.png


In [7]:
import math
import torch
import torch.nn.functional as F


def fisheye_to_equisolid_torch(
    img_bgr_uint8,      # torch.uint8 [Hf, Wf, 3] 或 [Hf, Wf]
    inv_poly_param,     # list/np.array, len = N (你的 OCamCalib inverse polynomial)
    affine_param,       # [c, d, e, xc, yc]
    W_out=1920,         # 输出等立体角图宽
    H_out=1920,         # 输出等立体角图高（建议方形以容纳完整圆盘）
    FOV_deg=180.0,      # 体视投影的视场（圆盘直径），边缘对应 FOV/2
):
    """
    将单张鱼眼图（OCamCalib 模型）重映射为等立体角(体视)投影平面图。
    返回 torch.uint8 的图，形状 [H_out, W_out, 3] 或 [H_out, W_out]。
    """
    # ---- 1) 准备输入图像张量 ----
    if img_bgr_uint8.dim() == 2:
        Hf, Wf = img_bgr_uint8.shape
        C = 1
        img = img_bgr_uint8.unsqueeze(0).unsqueeze(0).float()  # [1,1,Hf,Wf]
    else:
        Hf, Wf, _ = img_bgr_uint8.shape
        C = 3
        img = img_bgr_uint8.permute(2, 0, 1).unsqueeze(0).float()  # [1,3,Hf,Wf]

    device = img.device

    # ---- 2) 输出平面中心与等效焦距 f（等立体角）----
    cx = W_out / 2.0
    cy = H_out / 2.0
    r_edge = min(cx, cy)  # 圆盘的边界半径（像素）
    # FOV 定义为圆盘直径覆盖的视场 => 边缘对应 θ_edge = FOV/2
    theta_edge = math.radians(FOV_deg / 2.0)
    # 等立体角: r = 2 f sin(θ/2) => f = r_edge / (2 sin(θ_edge/2))
    f_eq = r_edge / (2.0 * math.sin(theta_edge / 2.0))

    # ---- 3) 生成输出平面像素网格 (u,v) 并换算到以中心为原点的坐标 ----
    u = torch.linspace(0, W_out - 1, W_out, device=device)
    v = torch.linspace(0, H_out - 1, H_out, device=device)
    uu, vv = torch.meshgrid(u, v, indexing="xy")  # [H_out, W_out]

    dx = uu - cx
    dy = vv - cy
    r = torch.sqrt(dx * dx + dy * dy)  # 到中心的半径

    # 方位角 φ
    phi = torch.atan2(dy, dx)  # [-pi, pi]

    # ---- 4) 半径->极角：r = 2 f sin(θ/2) => θ = 2 * asin(r/(2f))，圆外设为无效（黑色）----
    # 为了数值安全，先 clamp
    denom = 2.0 * f_eq
    r_norm = torch.clamp(r / denom, min=0.0, max=1.0)  # >1 的在圆外，稍后黑边
    theta = 2.0 * torch.asin(r_norm)  # [0, π]，当 r==r_edge 且 FOV=180° => θ=90°

    # ---- 5) 构造单位方向向量 [x,y,z]（以光轴为 z）----
    sin_theta = torch.sin(theta)
    cos_theta = torch.cos(theta)
    x_dir = sin_theta * torch.cos(phi)
    y_dir = sin_theta * torch.sin(phi)
    z_dir = cos_theta

    # ---- 6) 用与你原函数一致的 OCamCalib 角度定义求 rho(θ_ocam) ----
    # 原代码：M = sqrt(x^2 + y^2)，theta_ocam = -atan2(z, M)
    M = torch.sqrt(x_dir * x_dir + y_dir * y_dir)
    eps = 1e-6
    M_safe = torch.clamp(M, min=eps)
    theta_ocam = -torch.atan2(z_dir, M_safe)

    inv_poly = torch.tensor(inv_poly_param, dtype=x_dir.dtype, device=device)  # [N]
    # Horner 法：
    rho = torch.zeros_like(theta_ocam)
    for a in reversed(inv_poly):
        rho = rho * theta_ocam + a

    # ---- 7) 仿射映射到鱼眼像素 (u_f, v_f) ----
    c, d, e, xc, yc = [float(t) for t in affine_param]

    # 与原实现一致：将平面方向单位化后乘 rho
    scale = rho / M_safe  # ~= rho * (1 / sqrt(x^2+y^2))
    x_img = x_dir * scale
    y_img = y_dir * scale

    u_f = c * x_img + d * y_img + xc
    v_f = e * x_img +       y_img + yc

    # ---- 8) 归一化到 [-1,1] 并 grid_sample 采样 ----
    grid_x = (u_f / (Wf - 1)) * 2.0 - 1.0
    grid_y = (v_f / (Hf - 1)) * 2.0 - 1.0
    grid = torch.stack([grid_x, grid_y], dim=-1).unsqueeze(0)  # [1,H_out,W_out,2]

    # 圆外区域（r > r_edge 或 r_norm>1）我们直接标成越界值，使其变黑：
    # 最简单方式：保持 grid 值在 [-2, -2]（明显越界），从而 padding_mode="zeros" 给黑色。
    outside_mask = (r > r_edge + 1e-6)
    if outside_mask.any():
        mask4 = outside_mask.unsqueeze(0).unsqueeze(-1)  # [1,H_out,W_out,1]
        grid = torch.where(mask4, torch.full_like(grid, -2.0), grid)

    out_img = F.grid_sample(
        img, grid, mode="bilinear", padding_mode="zeros", align_corners=True
    )  # [1,C,H_out,W_out]

    # ---- 9) 输出为 uint8 HxWxC ----
    out_img = torch.clamp(out_img, 0, 255).round().byte()
    if C == 1:
        out = out_img.squeeze(0).squeeze(0)  # [H_out, W_out]
    else:
        out = out_img.squeeze(0).permute(1, 2, 0)  # [H_out, W_out, 3]
    return out


# ---------------- 使用示例 ----------------
if __name__ == "__main__":
    import cv2
    import numpy as np
    import torch

    inv_poly_param = [
        889.91582358, 528.72413142, -20.52846684, 67.73730056,
        48.02888563, -14.14051173, 20.22746961, 38.15325087,
        -6.98641569, -26.68100659, -12.41222046, -1.83261524,
    ]
    affine_param = [
        9.99949231e-01, -4.44517298e-04, 6.30993143e-04, 9.68732891e02, 7.48514758e02,
    ]

    # 读取 1920x1536 的鱼眼图 (BGR)
    fisheye = cv2.imread("./assets/SurCam01.jpeg", cv2.IMREAD_COLOR)
    assert fisheye is not None and fisheye.shape[1] == 1920 and fisheye.shape[0] == 1536

    img_t = torch.from_numpy(fisheye)  # [H,W,3], uint8, BGR

    # 输出等立体角图：建议方形画布容纳完整 180° 圆盘
    W_out, H_out, FOV = 1920, 1920, 200.0
    equisolid_bgr = (
        fisheye_to_equisolid_torch(
            img_t, inv_poly_param, affine_param,
            W_out=W_out, H_out=H_out, FOV_deg=FOV
        )
        .cpu()
        .numpy()
    )

    cv2.imwrite(f"equisolid_{W_out}x{H_out}_FOV{FOV}.png", equisolid_bgr)
    print(f"saved -> equisolid_{W_out}x{H_out}_FOV{FOV}.png")


saved -> equisolid_1920x1920_FOV200.0.png
